In [212]:
%load_ext autoreload
%autoreload 2

import dask
import dask.dataframe as dd
from distributed import Client
import glob
import pandas as pd
from pathlib import Path

from global_params import PATH_NGI_L2, NA_VALUES, NGI_L2_V, NGI_L2_R
from src.MT import getMTfromTime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [203]:
def peri_rows(orbdata):
    minalt = orbdata['alt'].min()
    peri_rows = orbdata[orbdata["alt"]==minalt].drop_duplicates()
    peri_rows = peri_rows[peri_rows["t_utc"] == peri_rows["t_utc"].min()]
    return peri_rows

In [210]:
tfile = "/home/marek/maven/data/sci/ngi/l2/2015/08/mvn_ngi_l2_csn-abund-16836_20150807T134137_v08_r01.csv"

In [211]:
cols = ["t_utc", "t_unix", "orbit", "alt", "lst", "long", "lat", "sza"]
df = pd.read_csv(tfile, usecols=cols)

In [206]:
newdf = peri_rows(df)
dtcol = pd.to_datetime(newdf["t_utc"], format='%Y-%m-%dT%H:%M:%S')
newdf["Ls"] = dtcol.apply(getMTfromTime)
newdf

,t_utc,t_unix,orbit,alt,lst,long,lat,sza,Ls
349,2015-08-07T15:13:12,1.438960e+09,1668,146.0152,21.9475,-141.1071,-74.29276,113.4748,24.119434


In [207]:
dtcol

349   2015-08-07 15:13:12
Name: t_utc, dtype: datetime64[ns]

In [264]:
tfiles = glob.glob(str(PATH_NGI_L2 / "*" / "*" / f"*v{NGI_L2_V}_r{NGI_L2_R}.csv" ))
tfiles = [x for x in tfiles if "mvn_ngi_l2_csn-abund-29514_20181107T071715_v08_r01.csv" not in x]

In [265]:
meta = {x: float for x in cols}
meta["t_utc"] = object
meta["orbit"] = int
ddf = dd.read_csv(tfiles, assume_missing=True, usecols=cols, dtype=meta, na_values=NA_VALUES)

In [266]:
print(ddf.npartitions)

8128


In [267]:
newdf = ddf.map_partitions(peri_rows)

In [268]:
dtcol = dd.to_datetime(newdf["t_utc"], format='%Y-%m-%dT%H:%M:%S')
newdf = newdf.assign(Ls=dtcol.apply(getMTfromTime, meta=float))

In [269]:
newdf.head(npartitions=10)

,t_utc,t_unix,orbit,alt,lst,long,lat,sza,Ls
330,2018-01-15T15:57:04,1.516032e+09,6408,139.5925,5.435278,-99.40232,-46.33352,111.9924,115.491251
341,2018-01-09T07:29:13,1.515483e+09,6374,140.3634,6.091389,-23.58109,-41.80220,104.3753,112.564029
321,2018-01-27T14:53:42,1.517065e+09,6472,139.4950,4.107500,11.27889,-54.66533,123.7902,121.058680
330,2018-01-22T13:50:52,1.516629e+09,6445,140.0549,4.689167,-12.69832,-51.12552,119.2895,118.700379
322,2018-01-09T11:58:19,1.515499e+09,6375,139.7262,6.072222,-89.35086,-41.94012,104.6123,112.649850


In [270]:
%%time
newdf = newdf.compute()

CPU times: user 7min 39s, sys: 31.4 s, total: 8min 10s
Wall time: 3min 4s


In [276]:
newdf.set_index("orbit").sort_index().to_csv("../output/peri.csv")